## CBOW 모델 구현

In [86]:
import sys
function_path = os.path.join("..","deep-learning-from-scratch-2-master")
chapter4_path = os.path.join(function_path, "ch04")
sys.path.append(function_path)
sys.path.append(chapter4_path)

from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

In [32]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V,H).astype("f")
        W_out = 0.01* np.random.randn(V,H).astype("f")
        
        # 계층 생성
        self.in_layers = []
        
        # 맥락개수만큼 in_layer개수 생성
        for i in range(2*window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
            
        # loss layer 생성
        self.ns_loss = NegativeSamplingLoss(W_out,corpus,power=0.75,sample_size=5)
        
        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
        
        # 인스턴스 변수에 단어의 분산 표현을 저장
        self.words_vecs = W_in
        
        def forward(self,contexts, target):
            h=0
            
            # in_layer 계산(각 층 계산후 평균값으로 h)
            for i, layer in enumerate(self,in_layers):
                h+= layer.forward(contexts[:,i])
            h *= 1/len(self.in_layers)
            
            # h를 이용하여 sample_size+1개의 뉴런에서의 loss 계싼
            loss = self.ns_loss.forward(h,target)
            
            return loss
        
        def backward(self, dout=1):
            dout = self.ns_loss.backward(dout)
            dout *= 1/len(self.in_layers)
            for layer in self.in_layers:
                layer.backward(dout)
            return None

## CBOW 모델 학습 코드

- 일반적으로 좋은 성능을 주는 하이퍼 파리미터
    - window_size : 2~10
    - 은닉층의 뉴런 수(분산 표현의 차원 수) : 50~100개

In [ ]:
import sys
function_path = os.path.join("..","deep-learning-from-scratch-2-master")
chapter4_path = os.path.join(function_path, "ch04")
sys.path.append(function_path)
sys.path.append(chapter4_path)

import numpy as np
from common import config

import pickle
from common.trainer import Trainer
from common.optimizer import Adam

from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data("train")
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)
    
# 모델 등 생성
model = CBOW(vocab_size, hidden_size , window_size, corpus)
optimizer = Adam()
trainer = Trainer(model,optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs # in_weight 꺼내오기
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {} # 필요한 파라미터 저장
params["word_vecs"] = word_vecs.astype(np.float16)
params["word_to_id"] = word_to_id
params["id_to_word"] = id_to_word

pkl_file = "cbow_params.pkl" # 파일명

# pkl_file 저장
with open(pkl_file, "wb") as f:
    pickle.dump(params, f,-1)

#### *pickle*
- list/class 같은 텍스트 형식이 아닌 자료형은 일반적인 방법으로 불러오거나 쓸 수 없음. -> pickle이 이를 제공
- 모든 데이터 객체를 저장하고 읽을 수 있음
- pickle로 데이터를 저장하거나 불러올때는 파일을 바이트형식으로 읽거나 써야함. (wb, rb)(원래는 w/r)
- wb로 데이터를 입력하는 경우는 .bin 확장자를 사용하는게 좋음
- 입력(pickle.dump(data, file))
    - list = [1,2,3,4]
    - with open('list.txt', 'wb') as f:  
        pickle.dump(list, f)
- 출력(pickle.load(file))
    - 한줄씩 읽어오고 더이상 읽어올 데이터가 없으면 Error 발생
    -  with open('list.txt', 'rb') as f:  
        data = pickle.load(f) 

## CBOW 모델 평가

In [44]:
weight = chapter4_path+"/cbow_params.pkl"

In [72]:
import sys
from common.util import most_similar
import pickle
pkl_file = "cbow_params.pkl"

with open(chapter4_path + "/"+pkl_file,"rb") as f:
    params = pickle.load(f)
    word_vecs = params["word_vecs"]
    word_to_id = params["word_to_id"]
    id_to_word = params["id_to_word"]

querys = ["you","year","car","toyota"]
for query in querys:
    most_similar(query,word_to_id,id_to_word,word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


## word2vec을 이용한 유추문제
- word2vec으로 얻은 단어의 분산 표현에서 비슷한 단어는 벡터공간에서 가깝게 위치. 더 복잡한 패턴 또한 파악함
- 벡터의 덧셈과 뺄셈을 이용하여 유추문제를 풀 수 있음
    - 단어의 단순한 의미(왕,남자,왕비,여자)뿐 아니라 문법적인 패턴(복수형, 과거/현재/미래형, 비교형 등)도 알 수 있음
- a:b=c:?
    - b와 a의 차이를 c에 더해줌으로써 ?를 구함
    - ? = b-a+c

In [48]:
def normalize(x):
    if x.ndim == 2:
        s = np.sqrt((x * x).sum(1))
        x /= s.reshape((s.shape[0], 1))
    elif x.ndim == 1:
        s = np.sqrt((x * x).sum())
        x /= s
    return x

In [80]:
def analogy(a,b,c, word_to_id, id_to_word,word_matrix, top=5, answer=None):
    for word in (a,b,c):
        if word not in word_to_id:
            print("%s(을)를 찾을 수 없습니다"%word)
            return
    
    # a:b=c:?
    # b와 a의 차이를 c에 더해줌으로써 ?를 구함
    print("\n[analogy]"+a+":"+b+"="+c+":?")
    a_vec, b_vec, c_vec = word_matrix[word_to_id[a]],word_matrix[word_to_id[b]], word_matrix[word_to_id[c]]
    query_vec = b_vec-a_vec+c_vec
    
    # 크기가 1이 되도록 정규화 시켜줌
    query_vec = normalize(query_vec)
    #word_matrix = normalize(word_matrix)
    
    # 코싸인 유사도
    similarity = np.dot(word_matrix,query_vec)
    
    if answer is not None:
        print("==>"+answer+":"+str(np.dot(word_matrix[word_to_id[answer]]), query_vec))
        
    count = 0
    
    # 유사도가 높은 것 순서대로 인덱싱 정렬
    for i in (-1*similarity).argsort():
        if np.isnan(similarity[i]): # nan 값이면
            continue
        if id_to_word[i] in (a,b,c): # a,b,c 안에 있는 단어면
            continue
        print("{0}:{1}".format(id_to_word[i],similarity[i]))
        
        count += 1
        if count >= top:
            return

In [85]:
analogy("king","man","queen",word_to_id, id_to_word, word_vecs, top=5) 
analogy("take","took","go",word_to_id, id_to_word, word_vecs, top=5)
analogy("car","cars","child",word_to_id, id_to_word, word_vecs, top=5)
analogy("good","better","bad",word_to_id, id_to_word, word_vecs, top=5)


[analogy]king:man=queen:?
arabs:0.459716796875
downside:0.455322265625
predecessor:0.44384765625
solution:0.434814453125
woman:0.433349609375

[analogy]take:took=go:?
went:0.47412109375
began:0.437255859375
starts:0.414794921875
turns:0.4130859375
comes:0.41259765625

[analogy]car:cars=child:?
children:0.50732421875
cattle:0.47412109375
sand:0.446533203125
screen:0.4375
sports:0.437255859375

[analogy]good:better=bad:?
slower:0.495361328125
more:0.48291015625
less:0.4609375
greater:0.432373046875
wider:0.4150390625


# word2vec 이용

### 단어의 분산 표현을 이용한 시스템 처리 흐름
질문(자연어) -> 단어 벡터화(word2vec) -> 머신러닝 시스템(NN/SVM 등) -> 답변

- 일반적으로 단어 분산 표현 학습과 머신러닝 시스템의 학습은 서로 다른 데이터셋을 사용해 개별적으로 수행
- 단어 벡터화
    - 단어/문장을 고정 길이 벡터로 변환할 수 있다는 것은 머신러닝 시스템의 입력으로 사용할 수 있고, 답을 출력하는 것이 가능해진다는 점에서 중요
    - 전이학습(한 분야에서 배운 지식을 다른 분야에도 적용하는 기법)을 이용
        - 큰 말뭉치(위키백과/구글 뉴스의 텍스트 데이터 등)으로 학습을 마친 분산표현을 각자의 작업에 이용
        - 이러한 단어의 분산표현을 이용하여 문장 또한 분산표현으로 나타낼 수 있음
            - 가장 간단한 방법(bag of words) : 문장의 각 단어를 분산 표현으로 변환하고 그 합을 구함, 순서를 고려하지 않음
            - RNN을 이용
- 머신러닝 시스템
    - 일반적으로 단어의 분산표현은 범용 말뭉치를 통해 미리 학습.
    - 머신러닝 시스템 학습은 현재 직면한 문제(텍스트 분류/문서 클러스터링/품사 태그달기/감성 분석 등)에 관련하여 수집한 데이터로 학습시킴
        - 직면한 문제의 학습 데이터가 아주 많다면 해당 데이터로 단어의 분산 표현, 머신러닝 시스템 학습 모두를 수행하는 방안 또한 고려해볼 수 있음

### 단어 벡터 평가 방법
- 실제 애플리케이션을 고려해서 평가
    - 자연어 처리 시스템은 단어분산표현과 머신러닝 시스템으로 구성
    - 현실적 문제 해결 관점에서 평가할 수 있음
    - but 두 시스템 모두를 고려하게 되면, 단어의 분산 표현의 차원에 따라 머신러닝 시스템의 최적의 하이퍼파라미터를 찾기 위한 튜닝도 필요하기 때문에, 실제 애플리케이션과는 분리해서 평가.
- 실제 애플리케이션과 분리해서 평가(일반적)
    - 평가 척도 : 유사성, 유추문제
    - 유사성 : 사람이 작성한 단어 유사도를 검증 세트로 사용해 평가
        - 사람이 부여한 점수와 word2vec에 의한 코사인 유사도 점수를 비교해 그 상관성을 봄
    - 유추 문제 : 의미(semantics), 구문(syntax) 등 평가
        - 모델에 따라 정확도가 다름(CBOW/skip-gram)
            - 말뭉치에 따라 적합한 모델 선택(일반적으로 skip_gram 성능이 더 좋음.)
        - 일반적으로 말뭉치가 클수록 결과가 좋음(항상 데이터가 많은 게 좋음)
        - 단어 벡터 차원 수는 적당한 크기가 좋음(너무 커도 정확도가 나빠짐)

- 단어 분산 표현의 우수함이 우수한 애플리케이션에 얼마나 기여하는지는 애플리케이션의 종류/말뭉치의 내용/다루는 문제 상황에 따라 다름
- 유츄 문제에 의한 평가가 높으면 애플리케이션에서도 좋은 결과를 기대할 수 있겠지만, 반드시 좋은 결과가 나오리라는 보장은 없음

## 정리
- 컴퓨터가 모든 데이터를 처리하는 것은 비현실적 -> 꼭 필요한 일부에 집중할 수 있도록 해줌